In [1]:
import msprime

In [2]:
# const population
demographic_events_const = [
    msprime.PopulationParametersChange(100, 1, growth_rate=0)
]

In [3]:
# one big growth
demographic_events_big = [
    msprime.PopulationParametersChange(50, 2500, growth_rate=0),
    msprime.PopulationParametersChange(100, 1, growth_rate=0)
]

In [4]:
# zig-zag
# one big growth
demographic_events_zig = [
    msprime.PopulationParametersChange(50, 2500, growth_rate=0.3),
    msprime.PopulationParametersChange(100, 1, growth_rate=0)
]

In [5]:
# bottle cheak
demographic_events_bc = [
    msprime.PopulationParametersChange(50, 1000, growth_rate=0),
    msprime.PopulationParametersChange(75, 5000, growth_rate=0),
    msprime.PopulationParametersChange(100, 1, growth_rate=0)
]

In [6]:
easy_pop = [
    demographic_events_const,
    demographic_events_big,
    demographic_events_zig,
    demographic_events_bc,
]

In [7]:
# out_of_africa

import math
def out_of_africa():
    # First we set out the maximum likelihood values of the various parameters
    # given in Table 1.
    N_A = 7300
    N_B = 2100
    N_AF = 12300
    N_EU0 = 1000
    N_AS0 = 510
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    T_AF = 220e3 / generation_time
    T_B = 140e3 / generation_time
    T_EU_AS = 21.2e3 / generation_time
    # We need to work out the starting (diploid) population sizes based on
    # the growth rates provided for these two populations
    r_EU = 0.004
    r_AS = 0.0055
    N_EU = N_EU0 / math.exp(-r_EU * T_EU_AS)
    N_AS = N_AS0 / math.exp(-r_AS * T_EU_AS)
    # Migration rates during the various epochs.
    m_AF_B = 25e-5
    m_AF_EU = 3e-5
    m_AF_AS = 1.9e-5
    m_EU_AS = 9.6e-5
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU and 2=CHB
    # initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=0, initial_size=N_AF),
        msprime.PopulationConfiguration(
            sample_size=1, initial_size=N_EU, growth_rate=r_EU),
        msprime.PopulationConfiguration(
            sample_size=1, initial_size=N_AS, growth_rate=r_AS)
    ]
    migration_matrix = [
        [      0, m_AF_EU, m_AF_AS],
        [m_AF_EU,       0, m_EU_AS],
        [m_AF_AS, m_EU_AS,       0],
    ]
    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        msprime.MassMigration(
            time=T_EU_AS, source=2, destination=1, proportion=1.0),
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        msprime.MigrationRateChange(time=T_B, rate=0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    # Use the demography debugger to print out the demographic history
    # that we have just described.
    dd = msprime.DemographyDebugger(
        population_configurations=population_configurations,
        migration_matrix=migration_matrix,
        demographic_events=demographic_events)
    # dd.print_history()
    
    return population_configurations, migration_matrix, demographic_events

In [8]:
import utilities.light_generator as ulg
import msprime


MU_HUMAN = ulg.give_mu()
RHO_HUMAN = MU_HUMAN * ulg.give_random_coeff()
POPULATION = 5000
num_replicates = 10
lengt = int(3e7)
is_experement = True

for demographic_events in easy_pop:
    
    dg = ulg.DataGenerator(
        recombination_rate=RHO_HUMAN,
        mutation_rate=MU_HUMAN,
        sample_size=POPULATION,
        demographic_events=demographic_events,
        num_replicates=num_replicates,
        lengt=lengt,
        random_seed=42,
        is_experement=is_experement)
    dg.run_simulation()
    
    print(1)
    data = next(dg)

1
1
1
1


In [9]:
population_configurations, migration_matrix, demographic_events = out_of_africa()


dg = ulg.DataGenerator(
    recombination_rate=RHO_HUMAN,
    mutation_rate=MU_HUMAN,
    demographic_events=demographic_events,
    population_configurations = population_configurations,
    migration_matrix = migration_matrix,
    num_replicates=num_replicates,
    lengt=lengt,
    random_seed=42,
    is_experement=is_experement)
dg.run_simulation()

data = next(dg)

In [10]:
data

(array([0, 0, 0, ..., 0, 0, 0]),
 [0.0,
  0.0010491333333333334,
  0.007317666666666667,
  0.0252635,
  0.07796913333333333,
  0.0807364,
  0.08254063333333334,
  0.062021266666666665,
  0.0752329,
  0.051672866666666664,
  0.050973833333333336,
  0.0972401,
  0.10754103333333333,
  0.0976238,
  0.08011243333333333,
  0.05707913333333333,
  0.030444633333333332,
  0.0121938,
  0.0025800333333333334,
  0.0003803666666666667,
  2.7333333333333335e-05],
 [145.38011839135496,
  200.68858815870664,
  277.038634049771,
  382.4353216111517,
  527.9291666935865,
  728.7747477707286,
  1006.0301011869465,
  1388.7645909660714,
  1917.1067414838349,
  2646.451588808227,
  3653.2686784486164,
  5043.11966006678,
  6961.72609909225,
  9610.247930968308,
  13266.374456576632,
  18313.43920430745,
  25280.61126177564,
  34898.37701367948,
  48175.129373963005,
  66502.89465577029])